In [273]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [99]:
colnames = ['date', 'time', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = pd.read_csv("anonymized-curriculum-access.txt", 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 1, 2, 3, 4, 5])
df['datetime'] = df.date + ' ' + df.time
df.datetime = pd.to_datetime(df.datetime)
df = df.set_index(df.datetime)
df = df.drop(columns = ['date', 'time', 'datetime'])
df.head()

/var/folders/8k/klwd85kj0d3_n66z9ht0xg7r0000gn/T/ipykernel_4400/2647315136.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("anonymized-curriculum-access.txt",


,endpoint,user_id,cohort_id,source_ip
datetime,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


# Question 4

### Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [100]:
df['day_of_week'] = df.index.day_name()

In [124]:
weekend_pulls = df.loc[
    (df.day_of_week == 'Saturday') | (df.day_of_week == 'Sunday')]

In [125]:
weekend_pulls.head()

,endpoint,user_id,cohort_id,source_ip,day_of_week
datetime,,,,,
2018-01-27 00:01:00,/,23,22.0,108.65.244.91,Saturday
2018-01-27 00:21:30,/,23,22.0,108.65.244.91,Saturday
2018-01-27 00:22:32,javascript-i/conditionals,23,22.0,108.65.244.91,Saturday
2018-01-27 00:39:16,spring/fundamentals/services,15,21.0,174.207.5.52,Saturday
2018-01-27 00:41:15,/,15,21.0,174.207.5.52,Saturday


In [131]:
weekend_pulls.groupby(['source_ip']).size().sort_values(ascending=False)

source_ip
192.171.117.210    2496
70.122.40.62        810
70.112.179.142      762
172.124.67.93       723
136.50.32.151       710
                   ... 
12.71.182.66          1
129.115.195.10        1
66.69.37.36           1
174.246.135.44        1
50.30.92.29           1
Length: 2181, dtype: int64

In [ ]:
sus_ip = df.loc[df.source_ip == '192.171.117.210']

In [144]:
len(sus_ip.user_id.unique())

306

In [145]:
len(sus_ip.cohort_id.unique())

32

The IP address '192.171.117.210' has tons of pulls from different users and cohorts, but this is not CodeUp's IP... This IP also has more pulls than any other on the weekends, so the only thing that would make sense is a place that students would gather on the weekend and access the cirriculum, like geekdom. I would like to raise a yellow flag here until we can confirm if this is Geekdom's IP. 

In [148]:
df.groupby(['user_id']).size().mean()

917.6585117227319

In [158]:
df.groupby(['user_id']).size().sort_values(
    ascending=False).head(25)

user_id
11     17913
64     16347
53     12329
314     7783
1       7404
495     6451
581     6434
428     5897
344     5460
248     5075
570     4584
315     4510
211     4227
423     3804
555     3687
545     3528
685     3522
524     3512
485     3268
671     3241
346     3182
566     3163
521     3052
595     3050
760     3019
dtype: int64

In [170]:
sus_user_314 = df.loc[df.user_id == 314]

In [174]:
len(sus_user_314)

7783

In [166]:
sus_user_314.cohort_id.value_counts()

28.0    4617
17.0    3166
Name: cohort_id, dtype: int64

In [167]:
sus_user_314.index.max() - sus_user.index.min()

Timedelta('815 days 11:04:22')

- This user has a cirriculum access time delta of 815 days between 2 cohorts, accesing the cirriculum almost 8000 times - the fourth most of any user. 

In [172]:
sus_user_53 = df.loc[df.user_id == 53]

In [173]:
sus_user_53.cohort_id.value_counts()

14.0    8197
28.0    4132
Name: cohort_id, dtype: int64

In [178]:
df.loc[df.user_id == 1].cohort_id.value_counts()

28.0    5787
8.0     1617
Name: cohort_id, dtype: int64

In [179]:
df.head()

,endpoint,user_id,cohort_id,source_ip,day_of_week
datetime,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Friday


In [180]:
df.endpoint.value_counts()

/                                                               50313
search/search_index.json                                        19519
javascript-i                                                    18983
toc                                                             18297
java-iii                                                        13733
                                                                ...  
javascript/loops                                                    1
content/control-structures-ii                                       1
javascript-ii/promisesdfghjkjhgfs                                   1
app                                                                 1
appendix/professional-development/post-interview-review-form        1
Name: endpoint, Length: 2313, dtype: int64

#### To answer this question, I decided to analyze activity outside of normal school days. After filtering the dataset into a new subset of instances on Saturdays and Sundays only, I found a particular IP that raises some suspicion. The IP address '192.171.117.210' has tons of access counts from different users and cohorts, but this is not CodeUp's IP. This IP has the most weekend access by a vast amount, so the only thing that would make sense is a place that students would gather on the weekend and access the curriculum, like geekdom. I would like to raise a yellow flag here until we can confirm if this is Geekdom's IP.

#### As for web scraping, I do not have enough information within the provided .txt file to begin analyzing if web scraping may be occurring on our curriculum. I certainly welcome the challenge of this task, but I will need information like the request header, so I can identify users not using traditional web browsers, as well as the file sizes being downloaded. With this information along with what I was provided, I am confident a proper analysis of this possibility can be conducted.

# Question 1

###  Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [194]:
# Group the data by endpoint and cohort_id to get the count of accesses
access_counts = df.groupby(["endpoint", "cohort_id"])["user_id"].count().reset_index()
access_counts.rename(columns={"user_id": "access_count"}, inplace=True)

# Calculate the average and standard deviation of access counts for each endpoint
endpoint_stats = access_counts.groupby("endpoint")["access_count"].agg([np.mean, np.std]).reset_index()
endpoint_stats.rename(columns={"mean": "avg_access_count", "std": "std_dev_access_count"}, inplace=True)

# Rank the endpoints by their standard deviation and find the one with the lowest standard deviation
endpoint_stats_sorted = endpoint_stats.sort_values("std_dev_access_count")
most_consistent_endpoint = endpoint_stats_sorted.iloc[0]["endpoint"]


In [208]:
access_counts = df.groupby(["endpoint", 
                            "cohort_id"])["user_id"].count().reset_index()

access_counts.rename(columns={"user_id": "access_count"}, inplace=True)

endpoint_stats = access_counts.groupby("endpoint")["access_count"].agg(
    [np.mean]).reset_index()

endpoint_stats.rename(columns={"mean": "avg_access_count"}, inplace=True)

endpoint_stats_sorted = endpoint_stats.sort_values("avg_access_count",
                                                  ascending=False)

endpoint_stats_sorted

,endpoint,avg_access_count
6,/,975.617021
1712,javascript-i,455.075000
2190,toc,439.775000
2007,search/search_index.json,427.658537
1016,classification/overview,376.714286
...,...,...
1274,content/mysql/aliases-and-functions/aliases.html,1.000000
1285,content/mysql/clauses/group-by.html,1.000000
1287,content/mysql/clauses/order-by.html,1.000000
1293,content/mysql/intro-to-mysql/index.html,1.000000


#### To answer this question, I created a subset of the data grouping by endpoint, or lesson, and cohort id. I then calculated the average access counts of all lessons within this subset and sorted to show the most accessed lesson on average among all the cohorts. I found that javascript I was accessed approximately 455 times on average, the highest for the web dev program. The highest accessed data science lesson was classification with approximately 377 access counts on average. 

# Question 2

### Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [210]:
endpoint_stats

,endpoint,avg_access_count
0,%20https://github.com/RaulCPena,1.0
1,',1.0
2,",%20https://github.com/RaulCPena",1.0
3,.git,1.0
4,.gitignore,1.0
...,...,...
2219,web-dev-day-two,2.0
2220,where,1.0
2221,working-with-time-series-data,7.5
2222,wp-admin,1.0


In [220]:
cohort_endpoint_stats = access_counts.groupby(["endpoint", "cohort_id"])["access_count"].agg(
    [np.sum]).reset_index()

cohort_endpoint_stats

,endpoint,cohort_id,sum
0,%20https://github.com/RaulCPena,55.0,1
1,',28.0,1
2,",%20https://github.com/RaulCPena",55.0,1
3,.git,24.0,1
4,.gitignore,24.0,1
...,...,...,...
13560,where,137.0,1
13561,working-with-time-series-data,28.0,1
13562,working-with-time-series-data,59.0,14
13563,wp-admin,22.0,1


In [224]:
sorted_endpoint = cohort_endpoint_stats.sort_values(
    'sum', ascending=False)

filtered_df = sorted_endpoint[sorted_endpoint["endpoint"] != "/"]

filtered_df.head(30)

,endpoint,cohort_id,sum
13351,toc,62.0,1866
8974,javascript-i,28.0,1817
11897,search/search_index.json,56.0,1497
13338,toc,29.0,1465
12460,spring,28.0,1403
8596,java-iii,28.0,1393
11891,search/search_index.json,33.0,1380
11887,search/search_index.json,28.0,1349
13345,toc,53.0,1293
1879,appendix,28.0,1288


In [227]:
spring_access = filtered_df.loc[filtered_df.endpoint == 'spring']
spring_access['sum'].mean()

304.6923076923077

In [228]:
spring_access['sum'].sum()

11883

In [230]:
len(spring_access)

39

In [233]:
(11883 - 1403)/38

275.7894736842105

#### To answer this question, I created a subset of the data grouping by endpoint, or lesson, and cohort id. I then calculated the sum of access counts by cohort id to see the total count of access of each lesson by cohort. I filtered out any access of the homepage, “/”, as this result does not reflect a lesson. I then displayed the table to review the top 30 most frequently accessed lessons by cohort to examine for a lesson that was heavily accessed by a single cohort. I found that the “spring” lesson was accessed a total of 1403 times by cohort 28, while the average access count of this lesson is approximately 305 per cohort. I also calculated the average access of this lesson with cohort 28 removed, and found the average access to this lesson is only ~276 per cohort. 

# Question 3

### Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [234]:
df.head()

,endpoint,user_id,cohort_id,source_ip,day_of_week
datetime,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Friday


In [275]:
access_per_user = df[['user_id', 'endpoint']].groupby(
    "user_id").count()
  
sorted_access = access_per_user.sort_values('endpoint')

sorted_access.describe()

,endpoint
count,981.000000
mean,917.657492
std,1208.871322
min,1.000000
25%,187.000000
50%,692.000000
75%,1297.000000
max,17913.000000


In [288]:
q1 = sorted_access.endpoint.quantile(.25)

low_access_count = sorted_access.loc[sorted_access.endpoint < q1]

low_access_count_users = low_access_count.index

low_access_count_users

Int64Index([652, 212, 879, 163, 952, 165, 649, 592, 593, 619,
            ...
            597, 938, 943,  17,  86, 175, 284,  57,  47, 932],
           dtype='int64', name='user_id', length=244)

In [334]:
filtered_df = df[df["user_id"].isin(low_access_count_users)]

cutoff_date = "2020-12-21"

filtered_df = filtered_df.loc[filtered_df.index <= cutoff_date]

filtered_df

,endpoint,user_id,cohort_id,source_ip,day_of_week
datetime,,,,,
2018-01-26 10:33:25,javascript-i/functions,13,22.0,97.105.19.61,Friday
2018-01-26 10:35:35,spring/extra-features/file-upload,17,21.0,97.105.19.61,Friday
2018-01-26 10:59:02,index.html,21,16.0,71.137.146.71,Friday
2018-01-26 10:59:06,java-i,21,16.0,71.137.146.71,Friday
2018-01-26 11:24:17,appendix,24,21.0,97.105.19.61,Friday
...,...,...,...,...,...
2020-12-20 18:10:03,javascript-i/conditionals,833,134.0,73.39.175.210,Sunday
2020-12-20 18:47:06,/,833,134.0,73.39.175.210,Sunday
2020-12-20 18:50:54,/,777,51.0,104.191.75.0,Sunday


In [340]:
filtered_df.user_id.nunique()

186

In [341]:
filtered_df.cohort_id.nunique()

39

In [339]:
low_users_by_cohort = filtered_df.groupby(
    "user_id").count().sort_values('endpoint')

low_user_count = low_users_by_cohort.cohort_id.value_counts().head(25)

low_user_count.loc[low_user_count.index > 10]

13     6
11     5
17     4
68     4
14     3
47     3
18     3
12     3
30     2
80     2
107    2
67     2
54     2
103    2
46     2
38     2
97     2
Name: cohort_id, dtype: int64

#### To answer this question, I created a subset of the data using only the user_id and endpoint columns, then grouped by user_id to get a count of accesses by user. I then calculated the 25th percentile of access count by user_id to isolate the users who hardly access the curriculum. I used 25th percentile as my qualifier for low access count due to the high standard deviation of this metric. Using any type of Interquartile Range Rule multiplier puts the lower fence in the negative, so this made the most sense for this analysis.

#### Once I had this list of users, I filtered the full dataframe to reproduce a new dataframe with these low access count users but removed any data from within the last 5 months to account for anyone who may possibly still be a current student. I found there are 186 students and 39 cohorts who had access counts in the 25th percentile. 

#### I then grouped this data by cohort_id to identify cohorts with access counts in the 25th percentile. I found that generally the cohorts identified here are the earlier cohorts, where much of the curriculum may not have been established yet. I further filtered out the first 10 cohorts, and found that cohorts 13, 11, 17, and 68 had the top 3 highest counts of low curriculum users with cohort 13 at 6, cohort 11 at 5, and cohorts 17 and 68 tied at 4. 


# Question 5

### Which lessons are least accessed?

In [342]:
df.head()

,endpoint,user_id,cohort_id,source_ip,day_of_week
datetime,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Friday
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Friday


In [385]:
endpoint_counts = df.endpoint.value_counts()

endpoint_counts.where(endpoint_counts == 1).value_counts()

1.0    457
Name: endpoint, dtype: int64

In [391]:
df.endpoint

datetime
2018-01-26 09:55:03                                                   /
2018-01-26 09:56:02                                             java-ii
2018-01-26 09:56:05                 java-ii/object-oriented-programming
2018-01-26 09:56:06                  slides/object_oriented_programming
2018-01-26 09:56:24                           javascript-i/conditionals
                                             ...                       
2021-04-21 16:41:51                                jquery/personal-site
2021-04-21 16:42:02                                   jquery/mapbox-api
2021-04-21 16:42:09                             jquery/ajax/weather-map
2021-04-21 16:44:37    anomaly-detection/discrete-probabilistic-methods
2021-04-21 16:44:39                                   jquery/mapbox-api
Name: endpoint, Length: 900223, dtype: object

In [431]:
df["endpoint"] = df["endpoint"].str.split("/").str[0]

endpoint_counts = df.groupby('endpoint').count().sort_values('user_id')

endpoint_counts

,user_id,cohort_id,source_ip,day_of_week
endpoint,,,,
wp-login,1,1,1,1
2.00.01_Navigating_Excel.md,1,1,1,1
extra,1,1,1,1
evaluate,1,1,1,1
end_to_end_clustering,1,1,1,1
...,...,...,...,...
spring,58603,55481,58603,58603
jquery,60869,58317,60869,60869
mysql,82320,79416,82320,82320


In [447]:
df["endpoint"] = df["endpoint"].str.replace('\d+', '')

/var/folders/8k/klwd85kj0d3_n66z9ht0xg7r0000gn/T/ipykernel_4400/3060483847.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["endpoint"] = df["endpoint"].str.replace('\d+', '')


In [452]:
df["endpoint"] = df["endpoint"].str.replace('-', '')

In [456]:
df["endpoint"] = df["endpoint"].str.replace('%', '')

In [460]:
df["endpoint"] = df["endpoint"].str.replace('.', '')

/var/folders/8k/klwd85kj0d3_n66z9ht0xg7r0000gn/T/ipykernel_4400/4047921949.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["endpoint"] = df["endpoint"].str.replace('.', '')


In [461]:
df["endpoint"] = df["endpoint"].str.replace('_', '')

In [462]:
df.endpoint.nunique()

583

In [463]:
(df.endpoint.value_counts() == 1).sum()

130

In [464]:
df.endpoint.value_counts() 

javascripti             118723
htmlcss                  84935
mysql                    82320
jquery                   60869
spring                   58603
                         ...  
filedat                      1
ui                           1
adacapsonteshtml             1
statisticsassessment         1
query                        1
Name: endpoint, Length: 583, dtype: int64

In [471]:
endpoint_counts = df.groupby('endpoint').count().sort_values('user_id')

endpoint_counts

,user_id,cohort_id,source_ip,day_of_week
endpoint,,,,
wplogin,1,1,1,1
filedat,1,1,1,1
f,1,1,1,1
extraexercises,1,1,1,1
EndtoEndclustering,1,1,1,1
...,...,...,...,...
spring,58603,55481,58603,58603
jquery,60869,58317,60869,60869
mysql,82320,79416,82320,82320


In [477]:
df.cohort_id.nunique()

47

In [478]:
greater_than_47 = endpoint_counts[endpoint_counts.user_id > 47]

In [479]:
greater_than_47[greater_than_47.user_id < 100]

,user_id,cohort_id,source_ip,day_of_week
endpoint,,,,
filepaths,49,11,49,49
DBSCANtodetectanomaliesmd,49,1,49,49
listingfiles,49,10,49,49
BuildingAModel,50,5,50,50
opendata,51,11,51,51
intro,53,9,53,53
Understand,54,29,54,54
navigatingthefilesystem,55,11,55,55
Inspect,55,11,55,55


#### To answer this question, I identified the value counts of all endpoints in the provided dataset. Out of 2313 unique endpoints, I found there were 457 instances where the end point was only accessed a single time. I edited this column to remove any text in the string after and including a “/” in an effort to isolate the data by lesson. I was then left with 755 unique endpoints, with 178 being accessed only once. There is no clear indication of what is a lesson or what is some other page a user ended up on based on the endpoint. However, I can be confident that if the endpoint was accessed less than the number of unique cohorts, it is likely not a lesson within the curriculum. When I filter out results where the endpoint was accessed less than 47 times,  and no more than 100, I’m left with a list of endpoints where there are clearly some lesson names to answer this question. I believe based on the data provided that the lessons least accessed are file paths, Intro to Classification, and HTML Forms. For full transparency, I am only concluding these results based on the .txt file provided that certainly lacks verbosity. I do not have much information to decipher from this data which of the endpoints are actually lessons. 